<h2>Calculate the Slope Classes for the Property</h2>

In this demo, we are tasked to provide a client the areas of their property that have a slope of:
- less than 15 degrees

We will be developing a web-based interactive map which can provide a better user experience for the client. 

First, let's load the slope raster and property boundaries, then reclassify the slope raster.

In [1]:
import fiona
import numpy as np
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import os

# Import geojson and clip raster
geojson_path = os.path.join('geojson', 'test_dataset_4326.geojson')
slope_raster = os.path.join('raster', 'slope_5m.tif')

# Load geojson using fiona
with fiona.open(geojson_path, 'r') as shape:
    shapes = [feature['geometry'] for feature in shape]

# Open slope raster and reclassify
with rasterio.open(slope_raster) as src:
    slope = src.read(1)
    slope[np.where(slope<15)] = 1
    slope[np.where(slope>=15)] = 2
    src_profile = src.profile.copy()

# Save to new file:
with rasterio.open(os.path.join('out', 'reclass.tif'), 'w', **src_profile) as dst:
    dst.write(slope, indexes=1)

# Open reclassified raster and clip to geojson
with rasterio.open(os.path.join('out', 'reclass.tif')) as src:
    slope, transform = mask(src, shapes, crop=True, all_touched=True, filled=True)
    new_profile = src.profile.copy()

# Update profile to dimensions of masked raster
new_profile.update({
    'width': slope.shape[2],
    'height': slope.shape[1],
    'transform': transform,
    'dtype': rasterio.uint8

})

# Save reclassified raster
with rasterio.open(os.path.join('out', 'reclass_masked.tif'), 'w', **new_profile) as dst:
    dst.write(slope)


Next, we will create rasters that represent each slope class. First is where the slope is less than 15 degrees.

In [2]:
# Create new raster where the slope value is equal to 1
with rasterio.open(os.path.join('out', 'reclass_masked.tif')) as src:
    slope = src.read(1)
    slope[np.where(slope!=1)] = 0
    src_profile = src.profile.copy()

    with rasterio.open(os.path.join('out', 'class1.tif'), 'w', **new_profile) as dst:
        dst.write(slope, indexes=1) 


Then the slope where slope is greater than or equal to 15 degrees. 

In [3]:
# Create new raster where the slope value is equal to 2
with rasterio.open(os.path.join('out', 'reclass_masked.tif')) as src:
    slope = src.read(1)
    slope[np.where(slope!=2)] = 0
    slope[np.where(slope==2)] = 1
    src_profile = src.profile.copy()

    with rasterio.open(os.path.join('out', 'class2.tif'), 'w', **new_profile) as dst:
        dst.write(slope, indexes=1) 

And a raster which represents the valid pixels, which represents the number of pixels available for each property boundary. This will be used to calculate the proportion of each class per boundary. 

In [4]:
# Create new raster containing the valid pixels
with rasterio.open(os.path.join('out', 'reclass_masked.tif')) as src:
    slope = src.read(1)
    slope[np.where(slope>0)] = 1
    src_profile = src.profile.copy()

    with rasterio.open(os.path.join('out', 'valid_pixels.tif'), 'w', **new_profile) as dst:
        dst.write(slope, indexes=1) 

Next, we will load the property boundaries.

In [6]:
# Read geojson using geopandas
property_boundaries = gpd.read_file(geojson_path)
property_boundaries = property_boundaries[['geometry', 'planlabel']]

And calculate the number of pixels for each class per boundary as well as the number of valid pixels.

In [7]:
# Open the rasters representating the classes and the valid pixels
class1_raster = rasterio.open(os.path.join('out', 'class1.tif'))
class2_raster = rasterio.open(os.path.join('out', 'class2.tif'))
valid_raster = rasterio.open(os.path.join('out', 'valid_pixels.tif'))

# Define a function that will calculate the statistic of the classes and the valid raster
def calc_stats_class1(geom, data=class1_raster, **mask_kw):
    masked, mask_transform = mask(dataset=data, shapes=(geom,),
                                  crop=True, all_touched=True, filled=True)
    return masked

def calc_stats_class2(geom, data=class2_raster, **mask_kw):
    masked, mask_transform = mask(dataset=data, shapes=(geom,),
                                  crop=True, all_touched=True, filled=True)
    return masked

def calc_stats_valid(geom, data=valid_raster, **mask_kw):
    masked, mask_transform = mask(dataset=data, shapes=(geom,),
                                  crop=True, all_touched=True, filled=True)
    return masked

# Calculate the pixel sum per class and the total for each boundary
# Generating a raster that represents valid pixels can serve as a better reference
# as it is based on the raster data and not the sum of two columns
property_boundaries['low_slope'] = property_boundaries.geometry.apply(calc_stats_class1).apply(np.sum)
property_boundaries['high_slope'] = property_boundaries.geometry.apply(calc_stats_class2).apply(np.sum)
property_boundaries['valid_pixels'] = property_boundaries.geometry.apply(calc_stats_valid).apply(np.sum)

property_boundaries.head()

,geometry,planlabel,low_slope,high_slope,valid_pixels
0,"MULTIPOLYGON (((149.02958 -32.55078, 149.02979...",DP835442,180066,22285,202351
1,"MULTIPOLYGON (((149.04284 -32.57281, 149.05208...",DP784596,54975,60563,115538
2,"MULTIPOLYGON (((149.05564 -32.56713, 149.05542...",DP827602,8365,17771,26136
3,"MULTIPOLYGON (((149.06405 -32.57903, 149.06322...",DP750779,2136,5778,7914
4,"MULTIPOLYGON (((149.06922 -32.5833, 149.06927 ...",DP875823,8755,13241,21996


Now, we can calculate the proportion of each class.

In [8]:
# Calculate the proportion for each class
property_boundaries['low_slope_proportion'] = round((property_boundaries['low_slope'] / property_boundaries['valid_pixels']) * 100, 2)
property_boundaries['high_slope_proportion'] = round((property_boundaries['high_slope'] / property_boundaries['valid_pixels']) * 100, 2)

property_boundaries.head()

,geometry,planlabel,low_slope,high_slope,valid_pixels,low_slope_proportion,high_slope_proportion
0,"MULTIPOLYGON (((149.02958 -32.55078, 149.02979...",DP835442,180066,22285,202351,88.99,11.01
1,"MULTIPOLYGON (((149.04284 -32.57281, 149.05208...",DP784596,54975,60563,115538,47.58,52.42
2,"MULTIPOLYGON (((149.05564 -32.56713, 149.05542...",DP827602,8365,17771,26136,32.01,67.99
3,"MULTIPOLYGON (((149.06405 -32.57903, 149.06322...",DP750779,2136,5778,7914,26.99,73.01
4,"MULTIPOLYGON (((149.06922 -32.5833, 149.06927 ...",DP875823,8755,13241,21996,39.80,60.20


Let's select the relevant columns for export.

In [9]:
# Select relevant columns
property_boundaries = property_boundaries[['geometry', 'planlabel', 'low_slope_proportion', 'high_slope_proportion']]
property_boundaries

,geometry,planlabel,low_slope_proportion,high_slope_proportion
0,"MULTIPOLYGON (((149.02958 -32.55078, 149.02979...",DP835442,88.99,11.01
1,"MULTIPOLYGON (((149.04284 -32.57281, 149.05208...",DP784596,47.58,52.42
2,"MULTIPOLYGON (((149.05564 -32.56713, 149.05542...",DP827602,32.01,67.99
3,"MULTIPOLYGON (((149.06405 -32.57903, 149.06322...",DP750779,26.99,73.01
4,"MULTIPOLYGON (((149.06922 -32.5833, 149.06927 ...",DP875823,39.80,60.20
5,"MULTIPOLYGON (((149.05969 -32.55221, 149.05933...",DP1213407,78.44,21.56
6,"MULTIPOLYGON (((149.06957 -32.57216, 149.07009...",DP827602,37.26,62.74


Lastly, we should convert the reclassified raster to vector for easier loading and viewing for the interactive web map.

In [10]:
from rasterio.features import shapes
from shapely.geometry import shape

# Convert the raster to geojson
with rasterio.open(os.path.join('out', 'reclass_masked.tif')) as src:
    data = src.read(1, masked=True)
    shape_gen = ((shape(z), x) for z, x in shapes(data, transform=src.transform))
    class_gdf = gpd.GeoDataFrame(dict(zip(["geometry", "class"], zip(*shape_gen))), crs=src.crs)


Save the files.

In [11]:
# Save files
property_boundaries.to_file(os.path.join('geojson', 'property_boundaries.geojson'), driver='GeoJSON')
class_gdf.to_file(os.path.join('geojson', 'class.geojson'), driver='GeoJSON')

And that's it! Go to `folium_map.ipynb` to generate the interactive web map.